# Ejercicio 1.a

In [1]:
import scipy
import matplotlib
import numpy
import random
!pip3 install seaborn
import seaborn as sns

In [2]:
modulo = 2**32
multiplicador = 1013904223
incremento = 1664525

semilla = (101503+103856+104258+102995)/4

In [3]:
print("Modulo: "+ str(modulo) )
print("Multiplicador: "+ str(modulo) )
print("Incremento: "+ str(incremento) )
print("Semilla: "+ str(semilla) )

Modulo: 4294967296
Multiplicador: 4294967296
Incremento: 1664525
Semilla: 103153.0


### Nuestra funcion de GCL quedaria como

X<sub>(i+1)</sub> = ( X<sub>(i)</sub> * Multiplicador + Incremento ) % modulo

In [4]:
lista = []

x = semilla
periodo = 0

while x not in lista:
    lista.append(x)
    periodo += 1
    x = (x * multiplicador + incremento) % modulo
    
print("El periodo es de "+ str(periodo) )

El periodo es de 12876


# Ejercicio 1.b

Para que nuestro GCL devuelva numero entre 0 y 1 podemos dividir por el modulo, normalizando el numero obtenido.

X<sub>(i+1)</sub> = ( X<sub>(i)</sub> * Multiplicador + Incremento ) % modulo

X<sub>(i+1)</sub> = X<sub>(i+1)</sub>  / modulo

In [5]:
x = semilla

for i in range(10):
    x = (x * multiplicador + incremento) % modulo
    print(x/modulo)

0.11999037768691778
0.6565184593200684
0.38245856761932373
0.832150936126709
0.3126610517501831
0.7375197410583496
0.0053147077560424805
0.638249876908958
0.527610182762146
0.4007291793823242


# Ejercicio 1.c

In [6]:
"""
Vuelvo a calcular los valores de lista de valores por mas de que quede calculada de antes.
"""

lista = []
listaNormalizada = []

x = semilla

for x in range(100000):
    lista.append(x)
    listaNormalizada.append(x/modulo)
    x = (x * multiplicador + incremento) % modulo